Calculate Tractor Trips and movements

- Get a list of all direction ideintfied images (mike and Neil)
- function
    - find the first unclaimed image
    - if mike to > find neil to within time period > give both same trip id > if cant mark as unclaimed
    - if mike from find neil from in time period and give same trip id > if cant mark as unclaimed
    
    - if neil to find mike to in time period > if not mark as unclaimed
    - if neil from find mike from in time period > if not mark as unclaimed
    
    - Result - all jined up trip
        - combine trip tin movements > math to and from trip in righ ttime period

    - unclaimed
        - match unclaimed neils as not via mike
        - should be many unclaimed mike      



In [7]:
#cameras
MIKE = 1
NEIL = 2

#direction
TO = 1
FROM = 0

#status
NOT_CLAIMED = 0
CLAIMED = 1
ORPHAN = 2

In [8]:
import pandas as pd
import psycopg2
import db_config as creds
import os
import time
import datetime
from datetime import datetime  
from datetime import timedelta  

from PIL import Image

In [9]:
class tractor:
  def __init__(self, filename, datetaken, cameraid,cameraname,direction):
    self.datetaken = datetaken
    self.filename = filename
    self.cameraname = cameraname
    self.cameraid = cameraid
    self.direction = direction
        
   

## Database Functions

In [10]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    #print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    #print ("disconnected!")

In [11]:
def getTractors():

    # Connecting to DB
    conn, cursor = connect()
    
    #get list of all tractor images with a direction
    dat = pd.read_sql(('select filename, datetaken_adjusted, cameraid, cameraname, true_direction from images '
                     'where datetaken BETWEEN %(dstart)s AND %(dfinish)s and true_direction is not null order by datetaken_adjusted asc'),
                   conn,params={"dstart":datetime(2020,5,8,0,0),"dfinish":datetime(2020,5,9,0,0)})

       
    dat["claimed"] = NOT_CLAIMED
    dat["tripid"] = 0
    dat["movementid"] = 0
    
    # Disconnect from DB
    disconnect(conn, cursor)
    
    print(dat)
    return dat

def findTrips(tractors):

    unclaimed = len(tractors[tractors['claimed'] == 0])
                    
    while (unclaimed > 0):
    
        #print(unclaimed) 
    
        #find next unclaimed tractor
        next_t = nextUnclaimed(tractors)
   
        #look for a related on either mike or neil           
        related_t = lookForRelated(next_t, tractors)
    
        #Get the next trip id
        tripid = tractors["tripid"].max() + 1
    
        #orphan, we cant find a matching trip image, set trip id, set status      
        if (next_t.filename == related_t.filename):
                    
            tractors.loc[tractors.filename == next_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
            #print(tractors.head())
        
        else:
        
            #update image
            tractors.loc[tractors.filename == next_t.filename, 'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
        
            #update related image  
            tractors.loc[tractors.filename == related_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == related_t.filename,'tripid'] = tripid 
    
        print("Original",next_t.filename,"Related to",related_t.filename)
        unclaimed = len(tractors[tractors['claimed'] == 0])
    else:
        print("trips mapped")      
    
#def findMovements(tractors)
    
    # try and link trip together 
    

def nextUnclaimed(tractors):

    #find the next tractor in the dataframe that is unclames
    t = tractors[tractors.claimed == 0].iloc[0]
    nextTractor = tractor(t[0],t[1],t[2],t[3],t[4])
   
    return nextTractor

def lookForRelated(t, tractors):  

    #print (t.filename)
    time_min, time_max = getTimeWindow(t)
    
    #mike
    if (t.cameraid == MIKE):
        
        # look for related neil to image 
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0) 
                             & (tractors.cameraid == NEIL)
                             & (tractors.true_direction == t.direction)
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
      

        
        except:
            print("No matching Neil image")
            related_t = t #just return the original
        
    else:    
        
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0)
                             & (tractors.cameraid == MIKE)   
                             & (tractors.true_direction == t.direction) 
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
        
        except:
            print("No matching mike image")
            related_t = t #just return the original
        
    return related_t
    
   
def getTimeWindow(t):
    
    #return min and max time range
    # it takes between 4 and 8 minutes to get between mike and neil 
    mins_min, mins_max = 4, 9
    
    #if mike to, then add time for neil
    if (t.cameraid == MIKE and t.direction == TO):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max)
    
    elif (t.cameraid == MIKE and t.direction == FROM):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == TO):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == FROM):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max) 
    
    
    #print("taken-",t.datetaken,"min-",minTime,"max-",maxTime)
    
    return minTime, maxTime    
    
    
def updateDb(tractors):
    
    # Connecting to DB
    conn, cursor = connect()
   
    sql = """UPDATE images SET trip_id2 = %s where filename = %s"""
  
    for index, row in tractors.iterrows():
        filename = row["filename"]
        tripid = row["tripid"] 
  
        cursor.execute(sql, (tripid,filename))
        conn.commit()
    
    print ("database updated")
    # Disconnect from DB
    disconnect(conn, cursor)
    

In [12]:
#get list of all tractors
df_tractors = getTractors()

#find trips
findTrips(df_tractors)
print(df_tractors)

#find movements = findMovements()
#findMovements(df_tractors)

#update database
updateDb(df_tractors)


         filename  datetaken_adjusted  cameraid cameraname  true_direction  \
0    14PT1460.JPG 2020-05-08 06:33:56         2       Neil               1   
1    04Mk2351.JPG 2020-05-08 06:36:48         1       Mike               1   
2    14PT1462.JPG 2020-05-08 06:43:20         2       Neil               1   
3    14PT1464.JPG 2020-05-08 06:45:52         2       Neil               0   
4    04Mk2355.JPG 2020-05-08 06:50:50         1       Mike               0   
..            ...                 ...       ...        ...             ...   
493  15PT1717.JPG 2020-05-08 21:47:20         2       Neil               0   
494  04Mk3851.JPG 2020-05-08 21:50:54         1       Mike               0   
495  04Mk3855.JPG 2020-05-08 21:51:00         1       Mike               0   
496  04Mk3859.JPG 2020-05-08 21:52:46         1       Mike               0   
497  15PT1719.JPG 2020-05-08 22:17:22         2       Neil               0   

     claimed  tripid  movementid  
0          0       0        

Original 04Mk3203.JPG Related to 15PT1592.JPG
Original 15PT1591.JPG Related to 04Mk3216.JPG
Original 04Mk3211.JPG Related to 15PT1593.JPG
Original 15PT1594.JPG Related to 04Mk3222.JPG
No matching mike image
Original 15PT1595.JPG Related to 15PT1595.JPG
Original 04Mk3225.JPG Related to 15PT1596.JPG
Original 04Mk3230.JPG Related to 15PT1601.JPG
No matching mike image
Original 15PT1597.JPG Related to 15PT1597.JPG
No matching Neil image
Original 04Mk3237.JPG Related to 04Mk3237.JPG
Original 15PT1598.JPG Related to 04Mk3246.JPG
Original 15PT1599.JPG Related to 04Mk3249.JPG
No matching mike image
Original 15PT1600.JPG Related to 15PT1600.JPG
No matching mike image
Original 15PT1602.JPG Related to 15PT1602.JPG
Original 04Mk3242.JPG Related to 15PT1603.JPG
Original 15PT1604.JPG Related to 04Mk3259.JPG
No matching mike image
Original 15PT1605.JPG Related to 15PT1605.JPG
Original 15PT1606.JPG Related to 04Mk3264.JPG
No matching mike image
Original 15PT1607.JPG Related to 15PT1607.JPG
No matching

databse updated
